<a href="https://colab.research.google.com/github/Andru-1987/74235-_DataScience_I/blob/main/clase_9/clase_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Asistentes de voces
* `pyttsx3`: is suitable for offline applications or when you need quick, local speech synthesis.
* `gTTS`: is ideal when you need high-quality, natural-sounding voices and have an internet connection.

In [ ]:
!pip install pyttsx3 gTTS beautifulsoup4 requests pillow pytesseract playsound

In [ ]:
import pyttsx3
import pytesseract
import requests
from gtts import gTTS
from PIL import Image
from bs4 import BeautifulSoup
import pygame
import io
import os
from abc import ABC, abstractmethod
from typing import Optional, Protocol


from IPython.display import Audio, display


In [ ]:
# Interfaces/Protocolos para inyección de dependencias
class TextToSpeechInterface(Protocol):
    """Interfaz para servicios de texto a voz"""
    def speak(self, text: str) -> None:
        """Texto a voz"""
        ...

class OCRInterface(Protocol):
    """Interfaz para servicios de OCR"""
    def extract_text(self, image_path: str) -> str:
        """Texto a imagen"""
        ...

class WebScrapingInterface(Protocol):
    """Interfaz para servicios de web scraping"""
    def extract_content(self, url: str) -> str:
        """Toma contenido web"""
        ...


In [ ]:
import sys
import io
from gtts import gTTS
from IPython.display import Audio, display

class PyttxGTTSTextToSpeech:
    """
    Implementación compatible con Google Colab.
    Si se detecta que está en Colab, usa gTTS + Audio de IPython.
    Si está en entorno local, intenta usar pyttsx3 (si está disponible).
    """

    def __init__(self, rate: int = 150, voice_index: int = 0, language: str = 'es'):
        self.language = language
        self.in_colab = 'google.colab' in sys.modules

        if not self.in_colab:
            try:
                import pyttsx3
                self.engine = pyttsx3.init()
                self.engine.setProperty("rate", rate)

                voices = self.engine.getProperty('voices')
                if voices and len(voices) > voice_index:
                    self.engine.setProperty('voice', voices[voice_index].id)
            except Exception as e:
                print(f"No se pudo inicializar pyttsx3: {e}")
                self.engine = None

    def speak(self, text: str) -> None:
        """Convierte texto a voz, compatible con Google Colab y entorno local."""
        try:
            if self.in_colab:
                # En Google Colab: usar gTTS
                tts = gTTS(text=text, lang=self.language)
                audio_buffer = io.BytesIO()
                tts.write_to_fp(audio_buffer)
                audio_buffer.seek(0)
                display(Audio(audio_buffer.read(), autoplay=True))
            elif hasattr(self, 'engine') and self.engine:
                # En entorno local con pyttsx3
                self.engine.say(text)
                self.engine.runAndWait()
            else:
                raise RuntimeError("No hay motor TTS disponible.")
        except Exception as e:
            print(f"Error en TTS: {e}")


class BeautifulSoupWebScraper:
    """Implementación de web scraping usando BeautifulSoup"""

    def __init__(self, timeout: int = 10, max_paragraphs: int = 3):
        self.timeout = timeout
        self.max_paragraphs = max_paragraphs
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def extract_content(self, url: str) -> str:
        """Extrae contenido de una página web"""
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer párrafos
            paragraphs = soup.find_all('p')
            text_content = []

            for p in paragraphs[:self.max_paragraphs]:
                text = p.get_text(strip=True)
                if text:  # Solo agregar párrafos no vacíos
                    text_content.append(text)

            return ' '.join(text_content)

        except requests.RequestException as e:
            raise Exception(f"Error al conectar con la URL: {e}")
        except Exception as e:
            raise Exception(f"Error al procesar el contenido: {e}")



### Clase especial para Imagen a Texto

In [ ]:
! lsb_release -a

In [ ]:
# Ubuntu/Debian
!sudo apt-get install tesseract-ocr tesseract-ocr-spa tesseract-ocr-eng espeak-ng

# # CentOS/RHEL
# !sudo yum install tesseract tesseract-langpack-spa tesseract-langpack-eng

# # Arch Linux
# !sudo pacman -S tesseract tesseract-data-spa tesseract-data-eng

# # Fedora
# !sudo dnf install tesseract tesseract-langpack-spa tesseract-langpack-eng

In [ ]:
import os
import platform
import shutil
from typing import Optional
from io import BytesIO

import pytesseract
import requests
from PIL import Image


class TesseractOCR:
    """Implementación de OCR usando Tesseract con soporte opcional de TTS y carga desde URL"""

    def __init__(self, language: str = 'spa+eng', config: str = '', tts_service: Optional[object] = None):
        self.language = language
        self.config = config
        self.tts_service = tts_service  # Objeto con método speak()

        self._configure_tesseract()
        self._verify_tesseract_installation()

    def _configure_tesseract(self):
        system = platform.system().lower()

        if system == 'linux':
            paths = [
                '/usr/bin/tesseract',
                '/usr/local/bin/tesseract',
                '/opt/tesseract/bin/tesseract',
                shutil.which('tesseract')
            ]
        elif system == 'windows':
            paths = [
                r'C:\Program Files\Tesseract-OCR\tesseract.exe',
                r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe',
                r'C:\tesseract\tesseract.exe'
            ]
        elif system == 'darwin':
            paths = [
                '/opt/homebrew/bin/tesseract',
                '/usr/local/bin/tesseract',
                shutil.which('tesseract')
            ]
        else:
            paths = [shutil.which('tesseract')]

        for path in paths:
            if path and os.path.exists(path):
                pytesseract.pytesseract.tesseract_cmd = path
                print(f"Tesseract encontrado en: {path}")
                return

        pytesseract.pytesseract.tesseract_cmd = 'tesseract'

    def _verify_tesseract_installation(self):
        try:
            version = pytesseract.get_tesseract_version()
            print(f"Tesseract version: {version}")
            languages = pytesseract.get_languages()
            print(f"Idiomas disponibles: {languages}")
        except Exception as e:
            print("Error: Tesseract no está instalado o no se encuentra en PATH")
            raise Exception(f"Tesseract no disponible: {e}")

    def extract_text(self, image_path: str) -> str:
        """Extrae texto de una imagen local o URL"""
        try:
            img = self._load_image(image_path)
            img = self._preprocess_image(img)

            text = pytesseract.image_to_string(
                img,
                lang=self.language,
                config=self._build_config()
            ).strip()

            if self.tts_service:
                self.tts_service.speak("Texto extraído de la imagen.")
                self.tts_service.speak(text)

            return text

        except Exception as e:
            raise Exception(f"Error al procesar la imagen: {e}")

    def extract_text_with_confidence(self, image_path: str) -> dict:
        """Extrae texto con información de confianza desde imagen local o URL"""
        try:
            img = self._load_image(image_path)
            img = self._preprocess_image(img)

            data = pytesseract.image_to_data(
                img,
                lang=self.language,
                config=self._build_config(),
                output_type=pytesseract.Output.DICT
            )

            confidences = [int(conf) for conf in data['conf'] if conf.isdigit()]
            avg_confidence = sum(confidences) / len(confidences) if confidences else 0
            text = ' '.join([word for word in data['text'] if word.strip()])

            if self.tts_service:
                self.tts_service.speak("Texto extraído con confianza.")
                self.tts_service.speak(text)

            return {
                'text': text,
                'confidence': avg_confidence,
                'word_count': len([word for word in data['text'] if word.strip()])
            }

        except Exception as e:
            raise Exception(f"Error al procesar imagen con confianza: {e}")

    def _load_image(self, path_or_url: str) -> Image.Image:
        """Carga una imagen desde ruta local o URL"""
        try:
            if path_or_url.startswith("http://") or path_or_url.startswith("https://"):
                response = requests.get(path_or_url)
                response.raise_for_status()
                return Image.open(BytesIO(response.content))
            else:
                if not os.path.exists(path_or_url):
                    raise FileNotFoundError(f"No se encontró la imagen: {path_or_url}")
                return Image.open(path_or_url)
        except Exception as e:
            raise Exception(f"No se pudo cargar la imagen: {e}")

    def _preprocess_image(self, img: Image.Image) -> Image.Image:
        width, height = img.size
        if width < 300 or height < 300:
            scale_factor = max(300 / width, 300 / height)
            img = img.resize((int(width * scale_factor), int(height * scale_factor)), Image.Resampling.LANCZOS)
        return img

    def _build_config(self) -> str:
        base = '--oem 3 --psm 6'
        return f"{base} {self.config}" if self.config else base

    def get_available_languages(self) -> list:
        try:
            return pytesseract.get_languages()
        except Exception as e:
            print(f"Error al obtener idiomas: {e}")
            return []


In [ ]:
# Clase principal del asistente con inyección de dependencias
class AsistenteVirtual:
    """Asistente virtual que utiliza inyección de dependencias"""

    def __init__(
        self,
        tts_service: Optional[PyttxGTTSTextToSpeech] = None,
        ocr_service: Optional[OCRInterface] = None,
        web_scraper: Optional[WebScrapingInterface] = None
    ):
        self.tts_service = tts_service
        self.ocr_service = ocr_service
        self.web_scraper = web_scraper

    def hablar(self, texto: str) -> None:
        """Hace que el asistente hable"""
        print("Asistente:", texto)
        self.tts_service.speak(texto)

    def texto_a_voz(self, texto: str) -> None:
        """Convierte texto a voz"""
        self.hablar(texto)

    def imagen_a_texto(self, path_imagen: str) -> None:
        """Extrae texto de una imagen y lo lee en voz alta"""
        if not self.ocr_service:
            self.hablar("Servicio OCR no disponible")
            return

        try:
            texto = self.ocr_service.extract_text(path_imagen)
            if texto:
                self.hablar("Texto extraído:")
                print(f"Texto extraído: {texto}")
                self.hablar(texto)
            else:
                self.hablar("No se pudo extraer texto de la imagen")
        except Exception as e:
            self.hablar("No pude procesar la imagen")
            print(f"Error: {e}")

    def extraer_info_web(self, url: str) -> None:
        """Extrae información de una página web y la lee"""
        if not self.web_scraper:
            self.hablar("Servicio de web scraping no disponible")
            return

        try:
            contenido = self.web_scraper.extract_content(url)
            if contenido:
                self.hablar("Información de la página:")
                print(f"Contenido extraído: {contenido}")
                self.hablar(contenido)
            else:
                self.hablar("No se pudo extraer contenido de la página")
        except Exception as e:
            self.hablar("Error al conectarme a la web")
            print(f"Error: {e}")


## Asistente de Voz

In [ ]:
asistente_voz = AsistenteVirtual(tts_service=PyttxGTTSTextToSpeech())
asistente_voz.hablar("Lulu menea la patita, menea la colita, mueve las alitas y se da una vueltecita")

In [ ]:
asistente_texto = AsistenteVirtual(
    ocr_service=TesseractOCR(language='spa+eng'),
    tts_service=PyttxGTTSTextToSpeech())

In [ ]:
# path_url = "https://s1.significados.com/foto/texto-expositivo-is-cke.jpg"
path_url = "https://es-static.z-dn.net/files/d09/5690a0badfd8c9544587825154a38380.png"
asistente_texto.imagen_a_texto(path_url)

In [ ]:
asistente_web = AsistenteVirtual(
    # ocr_service=TesseractOCR(language='spa+eng'),
    tts_service=PyttxGTTSTextToSpeech(),
    web_scraper=BeautifulSoupWebScraper()
    )


In [ ]:
asistente_web.extraer_info_web(url="https://en.wikipedia.org/wiki/Haversian_canal")